# Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import wrangle

from os.path import exists

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from importlib import reload

import warnings
#warnings.filterwarnings("ignore")


# Acquire

# Prep: Part IV 
* Goal:
    - Create functions that will handle sections of prep 
    - Big wrangle Function --> prep_aug()
    - don't forget to add docstring placeholders

In [32]:
def acquire_aug():
    ''' 
    Purpose:

    ---

    Parameters:

    ---

    Output:
    
    ---
    '''

    df = pd.read_csv('aug22pub.csv')

    df.columns = df.columns.str.lower()
    
    return df

# create function to flatten race in less categories
def flatten_race(val):
    ''' 
    Purpose:

    ---

    Parameters:

    ---

    Output:
    
    ---
    '''
    if val == 1:
        val = 'white'
    elif val == 2:
        val = 'black'
    elif val == 3:
        val = 'AI/NA'
    elif val == 4:
        val = 'asian'
    elif val == 5:
        val = 'HI/PI'
    elif 5 < val < 10:
        val = 'mixed_white'
    elif 10 < 27 :
        val = 'mixed_other'
    return val


def prep_values(df):
    ''' 
    Purpose:

    ---

    Parameters:

    ---

    Output:
    
    ---
    '''

    #decided to dropna's cleared up problem with target variable
    df = df.dropna()

    # fixing peafnow  --> get rid of peafnow
    df[df.peafnow == 1].loc[:,'peafever'] = 1

    #work done to fix usual_hours_worked with manual imputation 
    #mean hours worked for those less that work less than 35
    more_than_35 = round(df[df.pehruslt > 35].pehruslt.mean())
    #mean hours worked for those less that work less than 35
    less_than_35 = round(df[(df.pehruslt > 0) & (df.pehruslt < 35)].pehruslt.mean())
    df[['pehrftpt', 'pehruslt']].value_counts()

    df[df.pehrftpt == 1].loc[:,'pehruslt'] = more_than_35
    df[df.pehrftpt == 2].loc[:,'pehruslt'] = less_than_35

    # remove responses from people not in the labor force by means other that discouragement
    df = df[df.prempnot != 4]
    # remove responses from unqualified respondents (children and active armed forces members)
    df = df[df.prempnot != (-1)]
    # binary recode of premmpnot to include repsondents discourage from workforce participation as unemployed
    df.prempnot = np.where(df.prempnot == 1, 1, 0)

    return df


def prep_columns(df): 
    ''' 
    Purpose:

    ---

    Parameters:

    ---

    Output:
    
    ---
    '''

    df['ptdtrace'] = df['ptdtrace'].apply(flatten_race)

    final_list = ['hehousut', 'hefaminc', 'hrnumhou', 'hrhtype','hubus',\
    'gediv', 'gestfips', 'gtmetsta', 'gtcbsasz', 'prtage', 'pesex', \
    'pemaritl', 'peafever', 'peeduca', 'ptdtrace', 'pehspnon', 'penatvty', 'pemntvty',
    'pefntvty', 'prcitshp', 'pubus1', 'pehruslt', 'pelkavl', 'pedwlko', 'pedwwk', 'pejhwant', 'prempnot',\
    'prmjind1', 'prmjocc1', 'peschenr', 'prchld', 'pecert1']

    df = df[final_list]

    df = df.rename(columns={'hehousut': 'housing_type',
        'hefaminc': 'family_income',
        'hrnumhou': 'household_num',
        'hrhtype': 'household_type',
        'hubus': 'own_bus_or_farm',
        'gediv': 'country_region',
        'gestfips': 'state',
        'gtmetsta': 'metropolitan',
        'gtcbsasz': 'metro_area_size',
        'prtage': 'age',
        'pesex': 'is_male',
        'pemaritl': 'marital_status',
        'peafever': 'veteran',
        'peeduca': 'education',
        'ptdtrace': 'race',
        'pehspnon': 'hispanic_or_non',
        'penatvty': 'birth_country',
        'pemntvty': 'mother_birth_country',
        'pefntvty': 'father_birth_country',
        'prcitshp': 'citizenship',
        'pubus1': 'upaid_work_last_week',
        'pehruslt': 'usual_hours_worked',
        'prempnot': 'employed',
        'prmjind1': 'industry',
        'prmjocc1': 'occupation',
        'peschenr': 'enrolled_in_school',
        'prchld': 'children_in_household',
        'pecert1': 'professional_certification'})

    #fixing types on categorical columns
    categorical_cols = ['housing_type','family_income','household_type',
                        'country_region','state','metropolitan','metro_area_size',
                        'marital_status','education','race','birth_country',
                        'mother_birth_country','father_birth_country','citizenship',
                        'industry','occupation']

    binary_cols = ['own_bus_or_farm', 'is_male', 'veteran','hispanic_or_non', 
                'upaid_work_last_week','employed',
                'enrolled_in_school','professional_certification']  

    #for loop to handle assignment as object
    for col in categorical_cols:
        df[col] = (df[col].astype('object'))

    # changed all non-affirmative answers to negative. tips, school, most affected
    for col in binary_cols:
        df[col] = np.where(df[col] == 1, 1, 0)                                 

    return df

In [33]:
def prep_aug():
    ''' 
    Purpose:

    ---

    Parameters:

    ---

    Output:
    
    ---
    '''

    df = acquire_aug()

    df = prep_values(df)

    df = prep_columns(df)

    return df

In [36]:
reload(wrangle)

<module 'wrangle' from '/Users/sinao/coding_projects/oct_2022_unemployment_project/wrangle.py'>

# October Work

# Acquire  

In [2]:
df = pd.read_csv('oct22pub.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126649 entries, 0 to 126648
Columns: 388 entries, hrhhid2 to pternh2
dtypes: float64(337), int64(51)
memory usage: 374.9 MB


In [4]:
# list to hold columns for masking. 

In [5]:
columns_to_keep = [ 'hehousut',
                    'hrnumhou', 
                    'hefaminc',
                    'hrhtype',
                    'hubus',
                    'gediv',
                    'gestfips',
                    'gtmetsta',
                    'gtcbsasz',
                    'prtage',
                    'pemaritl',
                    'pesex',
                    'peafever',
                    'peeduca',
                    'ptdtrace', 
                    'pehspnon',
                    'penatvty',
                    'pemntvty',
                    'pefntvty', 
                    'prcitshp', 
                    'prinuyer',
                    'prempnot',
                    'prftlf',
                    'prcow1',
                    'prmjind1',
                    'prmjocc1', 
                    'peschenr', 
                    'prnmchld',
                    'peafwhn1',
                    'pecert1',\
                    
        ]

In [6]:
df.columns = df.columns.str.lower()

In [7]:
df = df[columns_to_keep]

In [8]:
df = df.rename(columns={'hehousut': 'housing_type',
        'hrnumhou': 'household_num',
        'hefaminc': 'family_income',
        'hrhtype': 'household_type',
        'hubus': 'own_bus_or_farm',
        'gediv': 'country_region',
        'gestfips': 'state',
        'gtmetsta': 'metropolitan',
        'gtcbsasz': 'metro_area_size',
        'prtage': 'age',
        'pemaritl': 'marital_status',
        'pesex': 'is_male',
        'peafever': 'veteran',
        'peeduca': 'education',
        'ptdtrace': 'race',
        'pehspnon': 'hispanic_or_non',
        'penatvty': 'birth_country',
        'pemntvty': 'mother_birth_country',
        'pefntvty': 'father_birth_country',
        'prcitshp': 'citizenship',        
        'prinuyer': 'immigration_era',
        'prempnot': 'employed',
        'prftlf': 'ft_or_pt',
        'prcow1': 'worker_class',
        'prmjind1': 'industry',
        'prmjocc1': 'occupation',
        'peschenr': 'enrolled_in_school',
        'prnmchld': 'num_children',
        'peafwhn1': 'service_era',
        'pecert1': 'professional_certification'})

# Prepare 

### drop any NILF(not in labor force or with -1)

In [9]:
df.employed.value_counts(0)

 1.0    48086
 4.0    33496
-1.0    17580
 2.0     1589
 3.0       88
Name: employed, dtype: int64

In [10]:
df = df[(df.employed == 1) | (df.employed == 2)].reset_index(drop=True)

### check for null values

In [11]:
df.isna()

,housing_type,household_num,family_income,household_type,own_bus_or_farm,country_region,state,metropolitan,metro_area_size,age,...,employed,ft_or_pt,worker_class,industry,occupation,enrolled_in_school,num_children,ged_or_diploma,service_era,professional_certification
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49670,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49671,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49672,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49673,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
df.household_num.value_counts()

2     16209
3      9881
4      9622
1      6358
5      4511
6      1905
7       692
8       278
9       129
10       36
11       35
12       12
13        7
Name: household_num, dtype: int64

### check for -1 values (those are the N/A type responses in the survey). Will try to impute them depeneding on the category

In [13]:
for col in df.columns:
    print(col)
    print(df[col].value_counts().sort_values())
    print('---')

housing_type
8         2
12       10
4        12
7        13
3        13
2        17
6       220
5      1556
1     47832
Name: housing_type, dtype: int64
---
household_num
13        7
12       12
11       35
10       36
9       129
8       278
7       692
6      1905
5      4511
1      6358
4      9622
3      9881
2     16209
Name: household_num, dtype: int64
---
family_income
3       246
2       285
5       384
1       426
4       466
6       725
7      1088
8      1251
9      1844
10     2084
11     3248
12     3748
13     5248
14     7342
15     9544
16    11746
Name: family_income, dtype: int64
---
household_type
10        4
9         8
8        16
5        17
2       189
3      3428
7      4805
4      6125
6      6162
1     28921
Name: household_type, dtype: int64
---
own_bus_or_farm
-1        1
-2       41
-3      119
 1     8518
 2    40996
Name: own_bus_or_farm, dtype: int64
---
country_region
6    3267
1    3976
2    4202
4    4825
3    5245
7    5470
8    6184
9    7377
5    

### takeaways
* columns to adjust
    own_bus_or_farm, veteran, unpaid_work_last_week, usual_35_or_more
* to drop, job_past_12_months, worker_class, industry, occupation, hourly, enrolled_in_school, ged_or_diploma, service_era
---

## imputing values and handling -1
* need to handle the -1 values by reading original data dictionary for guidance
* an answer not in the affirmative is treated as a negative for binary classification
    * own_bus_or_farm, veteran, service_era
* delete columns
    * unpaid, usual 35 or more, job past 12 months
* impute (decide on method) --> KNN impute
    * enrolled_in_school

In [14]:
df['own_bus_or_farm'] = df['own_bus_or_farm'].apply(lambda x: x == 1, 1,0)

In [15]:
df['veteran'] = df['veteran'].apply(lambda x: x == 1, 1,0)

In [16]:
df['service_era'] = df.service_era.apply(lambda x: 0 if x == -1 else x)

In [17]:
df.enrolled_in_school = df.enrolled_in_school.apply(lambda x: np.nan if x == -1 else x)

In [70]:
df[['worker_class', 'industry','occupation']] = df[['worker_class', 'industry','occupation']].applymap(lambda x: np.nan if x == -1 else x)

In [18]:
from sklearn.impute import KNNImputer

In [73]:
imputer = KNNImputer(n_neighbors=1)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

### Takeaways
* Nothing of major importance 
* was difficult understanding how to utilize the KNN imputer with the categorical nature of the column being imputed
---

## Flattening Columns

* Flattening is decreasing categorical optionality and renaming numerical entries
* Columns to flatten: 
 * housing_type, family income, household_type, marital_status, education, race, citzenship, birth country, mother birth country, father birth country, immigration era, service era

In [29]:
# create function to flatten race in less categories
def flatten_housing_type(val):
    if val == 1:
        val = 'perm_house_apt'
    elif 5 <= val <= 6:
        val = 'mobile_home'
    else:
        val = 'other'
    return val

In [30]:

def flatten_household_type(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val == 0:
        val = 'unknown'
    elif 1<= val <= 2:
        val = 'married'
    elif 3 <= val <= 8:
        val = 'unmarried'
    else:
        val = 'group'
    return val


In [31]:

def flatten_marital(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val in [1,2,5]:
        val = 'married'
    else:
        val = 'single'
    return val


In [36]:
def flatten_education(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val <= 38:
        val = 'no_high_school'
    elif val <= 40:
        val = 'high_school_ged'
    elif val < 43:
        val = 'associates'
    elif val == 43:
        val = 'bachelor'
    elif val > 43:
        val = 'post_grad'
    return val


In [37]:
# create function to flatten race in less categories
def flatten_race(val):
    if val == 1:
        val = 'white'
    elif val == 2:
        val = 'black'
    elif val == 3:
        val = 'AI/NA'
    elif val == 4:
        val = 'asian'
    elif val == 5:
        val = 'HI/PI'
    elif val in [6,7,8,9,16,17,18,19,20,21,23,24]:
        val = 'mixed_white'
    else :
        val = 'mixed_other'
    return val


In [33]:

def flatten_citizenship(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val <= 3:
        val = 'native'
    elif val == 4 :
        val = 'naturalized'
    else :
        val = 'foreign'

    return val


In [34]:

def flatten_immigration(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val == 57:
        val = 'us_50'
    elif val < 100:
        val = 'us_territories'
    else:
        val = 'foreign_country'
    return val


In [35]:
def flatten_immigration_era(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val <= 0:
        val = 'native'
    elif val < 17:
        val = 'pre_gwot'
    else:
        val = 'post_pwot'
    return val

In [61]:

def flatten_service_era(val):
    ''' 
    Purpose:
    To reduce the number of options  within the feature
    ---
    Parameters:
        val
    ---
    Output:
        val
    ---
    '''
    if val == 1:
        val = 'post_gwot'
    elif val < 1:
        val = 'pre_gwot'
    else:
        val = 'n/a'
    return val

In [41]:
df.housing_type = df.housing_type.apply(flatten_housing_type)

In [43]:
df.household_type = df.household_type.apply(flatten_household_type)

In [45]:
df.marital_status = df.marital_status.apply(flatten_marital)

In [25]:

df['race'] = df['race'].apply(flatten_race)

In [47]:
df.education = df.education.apply(flatten_education)

In [53]:
df.citizenship = df.citizenship.apply(flatten_citizenship)

In [56]:
df.birth_country = df.birth_country.apply(flatten_immigration)
df.mother_birth_country = df.mother_birth_country.apply(flatten_immigration)
df.father_birth_country = df.father_birth_country.apply(flatten_immigration)

In [59]:
df.immigration_era = df.immigration_era.apply(flatten_immigration_era)

In [63]:
df.service_era = df.service_era.apply(flatten_service_era)

## Renaming Column Values 
* Columns with values to be renamed
    * country_region, state, metropolitan, ft_or_pt, worker_class, industry, occupation
    * enrolled_in_school, 
* Binary Columns 
    * own_bus_or_farm, is_male, veteran, hispanic_or_non, employed, ged_or_diploma, professional_certification

In [74]:
for col in df.columns:
    print(col)
    print(df[col].value_counts().sort_values())
    print('---')

housing_type
other                67
mobile_home        1776
perm_house_apt    47832
Name: housing_type, dtype: int64
---
household_num
13.0        7
12.0       12
11.0       35
10.0       36
9.0       129
8.0       278
7.0       692
6.0      1905
5.0      4511
1.0      6358
4.0      9622
3.0      9881
2.0     16209
Name: household_num, dtype: int64
---
family_income
3.0       246
2.0       285
5.0       384
1.0       426
4.0       466
6.0       725
7.0      1088
8.0      1251
9.0      1844
10.0     2084
11.0     3248
12.0     3748
13.0     5248
14.0     7342
15.0     9544
16.0    11746
Name: family_income, dtype: int64
---
household_type
group           12
unmarried    20553
married      29110
Name: household_type, dtype: int64
---
own_bus_or_farm
1.0     8518
0.0    41157
Name: own_bus_or_farm, dtype: int64
---
country_region
6.0    3267
1.0    3976
2.0    4202
4.0    4825
3.0    5245
7.0    5470
8.0    6184
9.0    7377
5.0    9129
Name: country_region, dtype: int64
---
state
23.0   

In [75]:
region_map = {1: 'NEW ENGLAND', 2: 'MIDDLE ATLANTIC', 3: 'EAST NORTH CENTRAL', 4: 'WEST NORTH CENTRAL',
            5: 'SOUTH ATLANTIC',6: 'EAST SOUTH CENTRAL',7: 'WEST SOUTH CENTRAL', 8: 'MOUNTAIN',  9: 'PACIFIC' }

In [78]:
state_map={	1:'AL',2:'AK',
4:'AZ',
5:'AR' , 
6:'CA',
8:'CO', 
9:'CT', 
10:'DE',
11:'DC', 
12:'FL',
13:'GA',
15:'HI',
16:'ID',
17:'IL',
18:'IN',
19:'IA',
20:'KS',
21:'KY',
22:'LA',
23:'ME',
24:'MD',
25:'MA',
26:'MI',
27:'MN',
28:'MS',
29:'MO',
30:'MT',
31:'NE',
32:'NV',
33:'NH',
34:'NJ',
35:'NM',
36:'NY',
37:'NC',
38:'ND',
39:'OH',
40:'OK',
41:'OR',
42:'PA',
44:'RI',
45:'SC',
46:'SD',
47:'TN',
48:'TX',
49:'UT',
50:'VT',
51:'VA',
53:'WA',
54:'WV',
55 :'WI',
56:'WY',
}

In [82]:
metropolitan_map = {1:'metro', 2:'non_metro',3:'not_identified'}

In [85]:
ft_pt_map = {1: 'full_time', 2: 'part_time'}

In [93]:
round(df.worker_class.value_counts(),2)

4.000000    39356
5.000000     3307
3.000000     2994
2.000000     2480
1.000000     1429
3.820619       85
6.000000       24
Name: worker_class, dtype: int64

In [ ]:
worker_class_map = {1}

In [77]:
df.country_region = df.country_region.map(region_map)

In [81]:
df.state = df.state.map(state_map)

In [84]:
df.metropolitan = df.metropolitan.map(metropolitan_map)

In [87]:
df.ft_or_pt = df.ft_or_pt.map(ft_pt_map)

## Categorical vs Numerical

In [ ]:
#fixing types on categorical columns
categorical_cols = ['housing_type','family_income','household_type',
                    'country_region','state','metropolitan','metro_area_size',
                    'marital_status','education','race','birth_country',
                    'mother_birth_country','father_birth_country','citizenship',
                    'industry','occupation']

#for loop to handle assignment as object
for col in categorical_cols:
    df[col] = (df[col].astype('object'))

In [ ]:
#fixing binary columns freom -1, 1, 2 --> 0, 1
binary_cols = ['own_bus_or_farm', 'sex', 'veteran','hispanic_or_non', 
                'upaid_work_last_week','could_have_started_job',
                'sought_work_last_12_months', 'worked_last_12_months',
                'intend_to_look_next_12_months','employed', 'usual_ot_tips_commis',
                'enrolled_in_school','professional_certification']


In [ ]:
# changed all non-affirmative answers to negative. tips, school, most affected
# for col in binary_cols:
#     df[col] = np.where(df[col] == 1, 1, 0)